# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [9]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [10]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=['case/data=shakespeare', 
                                                      'case.model=transformer1'])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device(f'cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case single_imagenet with server type honest_but_curious.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [11]:
cfg.case.user.num_data_points = 21 # How many sentences?
cfg.case.user.user_idx = 0 # From which user?
cfg.case.data.shape = [32] # This is the sequence length

### Instantiate all parties

In [14]:
model, loss_fn = breaching.cases.construct_model(cfg.case.model, cfg.case.data, pretrained=True)
# Server:
server = breaching.cases.construct_server(model, loss_fn, cfg.case, setup)

## Modify the model here:

In [15]:
# Finalize changes:
model = server.vet_model(model)

In [17]:
# Construct the user here:
user = breaching.cases.construct_user(model, loss_fn, cfg.case, setup)

In [19]:
print(server)
print(user)

Server (of type HonestServer) with settings:
    Threat model: Honest-but-curious
    Number of planned queries: 1
    Has external/public data: False

    Model:
        model specification: transformer1
        model state: default
        public buffers: True

    Secrets: {}
    
User (of type UserSingleStep) with settings:
    Number of data points: 21

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: shakespeare
    user: 0
    
        


### Simulate an attacked FL protocol

True user data is returned only for analysis

In [20]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

In [21]:
user.print(true_user_data)

Yonder comes my master, your brother.But do not so. I have five hundred crowns,I scarce can speak to thank you for myself.

[Coming forward] Sweet masters, be patient; for your father'sremembrance, be at accord.
Is 'old dog' my reward? Most
 true, I have lost my teeth inCome not within these doors; within this roof
The enemy of all your graces lives.
Your brother- no
, no brother; yet the son-
Yet not the son; I will not call him son
Of him I was about to call his father-

Hath heard your praises; and this night he means
To burn the lodging where you use to lie,
And you within it. If he fail of
 that,
He will have other means to cut you off;
I overheard him and his practices.
This is no place; this house is but a
 butchery;
Abhor it, fear it, do not enter it.
No matter whither, so you come not here.O my sweet master
! O you memory
Of old Sir Rowland! Why, what make you here?
Why are you virtuous? Why do people love you?
And
 wherefore are you gentle, strong, and valiant?
Why would you 

# Reconstruct user data

In [ ]:
# Recover data 

In [ ]:
# Finally return a dict with keys data and labels
reconstructed_user_data = dict(data=None, labels=None)

In [ ]:
user.print(reconstructed_user_data)

### Check metrics:

In [ ]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)